# Import Solidity Source Code

In [ ]:
f = open("uts.sol", "r")
file_content = f.read()
f.close()

print(file_content)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

// Some codes here are not optimized for readability purpose
interface IERC20 {
    function approve(address spender, uint value) external;
    function transfer(address to, uint value) external;
    function transferFrom(address from, address to, uint value) external;
    function allowanced(address tokenOwner, address spender) external view returns (uint);
    function balanceOf(address user) external view returns (uint);
    function totalSupply() external view returns (uint);
}

// roughly 32 minutes for the basic ERC20
// Took 1 hour (+30 mins) to finish everything (with a little bit of looking up)
// +- 15 minutes for safeMath and testing
// +- 15 minutes for interface and bug fixing
contract erc20 is IERC20 {
    string public name = "AlfredKuhlmanToken";
    string public symbol = "AKT";
    uint public totalSupply = 1000;
    mapping(address => uint) allUser;
    mapping(address => mapping(address => uint)) allowance;
  

In [48]:
# For testing
file_content = """
// SPDX-License-Identifier: GPL-3.0
pragma solidity ^0.4.24;

contract something {
  uint public coolade;
  address public checker;
  address public owner;
  mapping(address => bool) public whitelist;
  int public more = 0;
  int256 public more3 = 23;
  int8 more2 = 0;
  uint16 more4;
  uint8 more5 = 0;
  uint[4] coolade2;
  int[][2] coolade3;
  int[2] coolade4;
  address[] fishe;
  string[] arr;
  bool bool1 = false;
  bool bool2 = true;
  string str1 = "";
  string str2 = "aa";
  string str3 = '';
  address address1 = 0x0000000000000000000000000000000000000000;
  address address2 = 0x0000000000000000000000000000000000000123;
  bytes4 bytesVar1 = "";
  bytes4 bytesVar2 = "abcd";
  bytes4 bytesVar3 = 0;
  bytes4 public bytesVar4 = '';
  int bool3;

  constructor() public {
    owner = msg.sender;
  }

  modifier onlyOwner() {
    require(msg.sender == owner);
    _;
  }

  function setChecker(address _checker) public onlyOwner {
    checker = _checker;
    for (int i = 0; i<10; i++) {
      more++;
    }

    for (int i; i<5; ++i) {
      ++more;
    }
  }

  function approve(address _wallet) public onlyOwner {
    whitelist[_wallet] = true;
  }
}
"""

print(file_content)


// SPDX-License-Identifier: GPL-3.0
pragma solidity ^0.4.24;

contract something {
  uint public coolade;
  address public checker;
  address public owner;
  mapping(address => bool) public whitelist;
  int public more = 0;
  int256 public more3 = 23;
  int8 more2 = 0;
  uint16 more4;
  uint8 more5 = 0;
  uint[4] coolade2;
  int[][2] coolade3;
  int[2] coolade4;
  address[] fishe;
  string[] arr;
  bool bool1 = false;
  bool bool2 = true;
  string str1 = "";
  string str2 = "aa";
  string str3 = '';
  address address1 = 0x0000000000000000000000000000000000000000;
  address address2 = 0x0000000000000000000000000000000000000123;
  bytes4 bytesVar1 = "";
  bytes4 bytesVar2 = "abcd";
  bytes4 bytesVar3 = 0;
  bytes4 public bytesVar4 = '';
  int bool3;

  constructor() public {
    owner = msg.sender;
  }

  modifier onlyOwner() {
    require(msg.sender == owner);
    _;
  }

  function setChecker(address _checker) public onlyOwner {
    checker = _checker;
    for (int i = 0; i<10; i++) {

In [2]:
import re

# Get contract version
#version = re.search('pragma solidity \^?(.*);', file_content).group(1)
# Get contract names
#name = re.findall('contract (.*) {', file_content)

# Utility functions
def swapPositionList(lst, pos1, pos2):
  lst[pos1], lst[pos2] = lst[pos2], lst[pos1]
  return lst

def swapPositionDict(dct, key1, key2):
  dct[key1], dct[key2] = dct[key2], dct[key1]
  return dct

def bitToByte(bit):
  return bit/8

In [50]:
### Process:
# Seperate source code to each lines -> list
# Scan and mark each lines based on its characteristics -> dict
# Optimize source code
# Combine all source code lines back

### Options
VARIABLE_PACKING = True
DEFAULT_VALUE = True
INCREMENT_OPERATOR = True

### Variable unpacking
# Scan for all variable islands
# Sort each variable islands from big to small byte size

### Seperate source code to each lines -> list
code_lines = file_content.split("\n")
code_metadata = {}

# Variables for detecting issues
all_vars = []  # Stores all islands of vars. Content: dict of byte sizes group
current_island_var = -1
last_index_var = -1  # To detect when the islands cut off.  if detected var > last_index_var+1
sorted_var = []

# 0: Symbol opening/closing, 1: major opener, 2: default value issue, 3: data types, 4: visibility, 5: operator
# Remember the ordering. First come, first serve
tags = {0: "{",
        1: "}",
        100: "contract ",
        101: " function ",
        102: " modifier ",
        103: " constructor",
        201: "bool(.*?) = false",  # Default Values (bool)
        202: "(u?)int(.*?) = 0",  # Default Values (int)
        203: "string(.*?) = (('')|(\"\"))",  # Default Values (string)
        204: "address(.*?) = 0x0000000000000000000000000000000000000000",  # Default Values (address)
        205: "bytes(.*?) = ((0(x(0+))?)|('')|(\"\"))",  # Default Values (bytes)
        301: " bool((\[([0-9]+)?\])+)? ",
        302: " int([0-9]+)?((\[([0-9]+)?\])+)? ",  # old: " int(.*?) "
        303: " uint([0-9]+)?((\[([0-9]+)?\])+)? ",
        304: " bytes([0-9]+)?((\[([0-9]+)?\])+)? ",
        305: " address((\[([0-9]+)?\])+)? ",
        306: " mapping\(",  # Variable length
        307: " string((\[([0-9]+)?\])+)? ",  # Variable length
        401: " public ",
        402: " private ",
        403: " external ",
        501: " for( |\()?",
        502: "\+\+( |\)|;)",
        }  # token: tag

def add_variable_island(byte_size, dict_line):
  global last_index_var
  global current_island_var
  global all_vars
  global tags

  # Ignore if it's a function / modifier / similar types
  ignore_case = (101, 102, 103)
  for i in ignore_case:
    if re.search(tags[i], dict_line["value"]) != None:
      return

  # Start new island
  if last_index_var+1 < dict_line["index"]:
    current_island_var += 1
    all_vars.append({})
    all_vars[current_island_var]["metadata"] = {}
    all_vars[current_island_var]["metadata"]["start_index"] = dict_line["index"]
    all_vars[current_island_var]["metadata"]["total_line"] = 0
    all_vars[current_island_var]["metadata"]["byte_sizes"] = set()

  # Add to island var
  if last_index_var != dict_line["index"]:  # Avoid the same line twice
    all_vars[current_island_var].setdefault(byte_size, []).append(dict_line)
    all_vars[current_island_var]["metadata"]["total_line"] += 1
    all_vars[current_island_var]["metadata"]["byte_sizes"].add(byte_size)
    last_index_var = dict_line["index"]

### Scan and mark each lines based on its characteristics -> dict
for index, value in enumerate(code_lines):
  metadata = [value]
  for token in tags:

    # https://pynative.com/python-regex-pattern-matching/
    # Check each code line for tag tokens
    if re.search(tags[token], value) != None:
      metadata += [token]
      _line_data = {"index": index, "value": value, "token": token}

      ## Default value fix
      if DEFAULT_VALUE == True:
        if token == 201:
          print(index, "index, default value bool issue found!!", value)

          fixed = re.sub(" = false", "", value)
          print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          code_lines[index] = fixed
        elif token == 202:
          print(index, "index, default value int issue found!!", value)

          fixed = re.sub(" = 0", "", value)
          print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          #_line_data["value"] = fixed
          code_lines[index] = fixed
        elif token == 203:
          print(index, "index, default value string issue found!!", value)

          fixed = re.sub(" = (('')|(\"\"))", "", value)
          print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          code_lines[index] = fixed
        elif token == 204:
          print(index, "index, default value address issue found!!", value)

          fixed = re.sub(" = 0x0000000000000000000000000000000000000000", "", value)
          print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          code_lines[index] = fixed
        elif token == 205:
          print(index, "index, default value bytes issue found!!", value)

          fixed = re.sub(" = ((0(x(0+))?)|('')|(\"\"))", "", value)
          print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          code_lines[index] = fixed


      ## Scan variable islands
      if VARIABLE_PACKING == True:
        if token == 301:  # bool
          print(index, "index, bool detected")

          # Check byte size
          size = re.search(r' bool(.*?) ', value).group(1)
          if size == "":
            byte_size = 1
          elif size.isdecimal() == False:
            byte_size = 33   # Variable size (array) is set to 33 by default because they can take up more than one page
          add_variable_island(byte_size, _line_data)  # Added the token variable to know what kind of var this is.

        elif token == 302:  # int
          print(index, "index, int detected")

          # Check byte size
          size = re.search(r' int(.*?) ', value).group(1)
          if size == "":
            size = 256
          elif size.isdecimal() == False:
            # Check int bit size
            _temp_split = size.split("[")
            if _temp_split[0].isdecimal() == False:
              size = 256
            else:
              size = 264
          byte_size = int(size)//8
          add_variable_island(byte_size, _line_data)

        elif token == 303:  # uint
          print(index, "index, uint detected")

          # Check byte size
          size = re.search(r' uint(.*?) ', value).group(1)
          if size == "":
            size = 256
          elif size.isdecimal() == False:
            # Check int bit size
            _temp_split = size.split("[")
            if _temp_split[0].isdecimal() == False:
              size = 256
            else:
              size = 264
          byte_size = int(size)//8
          add_variable_island(byte_size, _line_data)

        elif token == 304:  # bytes
          print(index, "index, bytes detected")

          # Check byte size
          byte_size = re.search(r' bytes(.*?) ', value).group(1)
          if byte_size.isdecimal() == False:
            _temp_split = byte_size.split("[")
            if _temp_split[0] == "32":
              byte_size = 32
            else:
              byte_size = 33
          add_variable_island(int(byte_size), _line_data)

        elif token == 305:  # address
          print(index, "index, address detected on index")

          # Check byte size
          size = re.search(r' address(.*?) ', value).group(1)
          if size == "":
            byte_size = 20
          elif size.isdecimal() == False:
            byte_size = 33
          add_variable_island(byte_size, _line_data)

        elif token == 306:  # mapping
          print(index, "index, mapping detected")
          add_variable_island(33, _line_data)  # Note: This is very variable, so take this with heavy grain of salt on its efficiency

        elif token == 307:  # string
          print(index, "index, string detected")
          add_variable_island(32, _line_data)  # Observed that setting this to 32 bytes is more gas efficient than 33


      ## Unoptimize operator use
      if INCREMENT_OPERATOR == True:
        if token == 502:  # x++
          print(index, "index, unoptimal x++ use detected", value)

          # Get variable name
          left_wall_var = " |\(|\)|;|<|>|="
          line_split = value.split("++")
          for i in range(len(line_split)-1):
            # Cut from the right edge until hits the left wall
            var_name = re.split(left_wall_var, line_split[i])[-1]

            if var_name == "":
              print("Empty found")
              continue

            fixed = value.replace(var_name+"++", "++"+var_name)

            value = fixed
            metadata[0] = fixed
            code_lines[index] = fixed
            print("Fixed to:", fixed)




  code_metadata[index] = metadata

### Optimize source code
print("")
# Variable optimization
print("Island count:", len(all_vars))
for index, value in enumerate(all_vars):
  # Uint* vs Uint256 - may affect execution cost. Halted before proper research
  for i in value["metadata"]["byte_sizes"]:
    print("Size:", i, "Count:", len(value[i]))
    if len(value[i]) == 1:
      print("1 length detected")  # Convert to uint256 / int256 then change the byte size to 256

      # Check what type of var this is.


  ## Variable Packing
  # Sort the variables based on their byte size
  sorted_var = []
  for i in sorted(value["metadata"]["byte_sizes"], reverse=True):
    sorted_var.extend(value[i])

  print("Sorted vars:", sorted_var)

  # Organize source code to be sorted
  #for idx, val in enumerate(sorted_var):  # Copy the source code in order
  #  sorted_var[idx] = code_lines[val]
  for idx, val in enumerate(sorted_var):  # Paste the source code to the real list
    code_lines[value["metadata"]["start_index"]+idx] = val["value"]

print("")

### Combine all source code lines back
code_optimized = '\n'.join(code_lines)

print("all_vars:", all_vars)
print("code_optimized:", code_optimized)
code_metadata

5 index, uint detected
6 index, address detected on index
7 index, address detected on index
8 index, mapping detected
9 index, default value int issue found!!   int public more = 0;
Fixed to:   int public more;
9 index, int detected
10 index, int detected
11 index, default value int issue found!!   int8 more2 = 0;
Fixed to:   int8 more2;
11 index, int detected
12 index, uint detected
13 index, default value int issue found!!   uint8 more5 = 0;
Fixed to:   uint8 more5;
13 index, uint detected
14 index, uint detected
15 index, int detected
16 index, int detected
17 index, address detected on index
18 index, string detected
19 index, default value bool issue found!!   bool bool1 = false;
Fixed to:   bool bool1;
19 index, bool detected
20 index, bool detected
21 index, default value string issue found!!   string str1 = "";
Fixed to:   string str1;
21 index, string detected
22 index, string detected
23 index, default value string issue found!!   string str3 = '';
Fixed to:   string str3;
2

{0: [''],
 1: ['// SPDX-License-Identifier: GPL-3.0'],
 2: ['pragma solidity ^0.4.24;'],
 3: [''],
 4: ['contract something {', 0, 100],
 5: ['  uint public coolade;', 303, 401],
 6: ['  address public checker;', 305, 401],
 7: ['  address public owner;', 305, 401],
 8: ['  mapping(address => bool) public whitelist;', 306, 401],
 9: ['  int public more;', 202, 302, 401],
 10: ['  int256 public more3 = 23;', 302, 401],
 11: ['  int8 more2;', 202, 302],
 12: ['  uint16 more4;', 303],
 13: ['  uint8 more5;', 202, 303],
 14: ['  uint[4] coolade2;', 303],
 15: ['  int[][2] coolade3;', 302],
 16: ['  int[2] coolade4;', 302],
 17: ['  address[] fishe;', 305],
 18: ['  string[] arr;', 307],
 19: ['  bool bool1;', 201, 301],
 20: ['  bool bool2 = true;', 301],
 21: ['  string str1;', 203, 307],
 22: ['  string str2 = "aa";', 307],
 23: ['  string str3;', 203, 307],
 24: ['  address address1;', 204, 305],
 25: ['  address address2 = 0x0000000000000000000000000000000000000123;', 305],
 26: ['  by

## The Meat (https://regexr.com/)
### Regex
### https://github.com/andhikan207/SC_TestBench/blob/main/Benchmarking.ipynb
### https://pynative.com/python-regex-pattern-matching/
### https://www.regextutorial.org/regex-for-numbers-and-ranges.php
### Optimization Tips
### https://www.linkedin.com/pulse/solidity-gas-golfing-1-uint8-vs-uint256-sudeep-sagar/
### https://dev.to/javier123454321/solidity-gas-optimizations-pt-3-packing-structs-23f4

In [ ]:
from IPython.display import clear_output
import pandas as pd
import re

#File name
file_name = "dummyfile.csv"

In [ ]:
#Load smart contracts list.
df = pd.read_csv(file_name)

#Convert dataframe to list.
smartContracts = []
smartContracts2 = []
smartContracts = df['Source Code'].tolist()

#Process to add keywords to individual smart contracts.
unfinishedKW = "UNFINISHED_SC"
failedKW = "NOT_A_SC"

#Manually input keywords
numCycle1 = 1

for i in smartContracts:
  clear_output(wait = True)
  print(i + "\n")

  while True:
    print(f"{numCycle1}. Please check this smart contract. (1. Finished, 2. Unfinished, 3. Failed)")
    scCondition = input("Input a number: ")

    if scCondition == "1":
      smartContracts2.append(i)
      numCycle1 += 1
      break
    elif scCondition == "2":
      smartContracts2.append(unfinishedKW + " " + i)
      numCycle1 += 1
      break
    elif scCondition == "3":
      smartContracts2.append(failedKW + " " + i)
      numCycle1 += 1
      break
    else:
      print("\n> Wrong input. Try again!")

clear_output(wait = True)
print("Done, please proceed to the next step!")

Done, please proceed to the next step!


In [ ]:
#Choose ERC Criteria
clear_output(wait = True)
chooseERC = ""

while True:
  print("1. ERC-20")
  print("2. ERC-4626")
  print("3. ERC-3156")
  chooseERC = input("Which scenario would you choose?: ")

  if chooseERC == "1" or chooseERC == "2" or chooseERC == "3":
    break
  else:
    print("\n> Wrong input. Try again!")


#Set criteria weight
clear_output(wait = True)

cWeight1 = 0
cWeight2 = 0
cWeight3 = 0
cWeight4 = 0
cWeight5 = 0
cWeight6 = 0
cWeight7 = 0
cWeight8 = 0
cWeight9 = 0
cWeight10 = 0

criteriaDesc = ["Complete source code", "Packing variable", "Packing booleans", "Storage type", "Data type", "Variable size type", "Default value", "Functions", "Storage limiting", "Minimize on-chain data"]
cWeights = [cWeight1, cWeight2, cWeight3, cWeight4, cWeight5, cWeight6, cWeight7, cWeight8, cWeight9, cWeight10]

while True:
  totalPerc = 0
  numCycle2 = 1
  cArr = 0

  for i in criteriaDesc:
    while True:
      try:
        print(f"{numCycle2}. Please write score weight in PERCENTAGE for {i}. (Currently {totalPerc}%/100%)")
        setWeights = float(input("(Numbers only): "))

        cWeights[cArr] = 1 * (setWeights / 100)

        cArr += 1
        numCycle2 += 1
        totalPerc += setWeights
        clear_output(wait = True)
        break
      except ValueError:
        print("\n> That is not a number!")

  totalWeight = 0
  for i in cWeights:
    totalWeight += i

  if round(totalWeight) != 1.0:
    print("> Weight exceed 1.0 or below!\n")
    continue
  else:
    clear_output(wait = True)
    cArr = 0

    print("[Criteria Weight]\n")

    for i in criteriaDesc:
      print(f"{i}: {cWeights[cArr]}")
      cArr += 1

    print(f"\nTotal weight: {round(totalWeight)}\n")
    weightDone = input("Proceed? y/n: ")

    if weightDone == "n":
      clear_output(wait = True)
      continue
    else:
      break

clear_output(wait = True)
print("Done, please proceed to the next step!")

Done, please proceed to the next step!


In [ ]:
#Benchmarking
def benchmarkProgram():
  #REGEX PATTERNS
  criteria1_UFSC = unfinishedKW #Unfinished Source Code
  criteria2_ZONK = failedKW #Failed Source Code
  notPacked = r"(uint|uint(256))+ public [A-Za-z0-9]+ = [0-9]+;(\n)+\s+uint(8|16|32|128)" #Packing variable
  isBoolean = r"(uint|uint[0-9])+\([0-9]\)" #Packing boolean

  #Storage Type
  possibleC1_V1 = "uint"
  possibleC1_V2 = "uint256"
  pat7 = r"uint(?!(?:256))\d+ public" # uint*

  pat1 = r"mapping\([^)]*\) public [A-Za-z0-9]+;" #Mapping
  pat3 = r"[A-Za-z0-9]+\[\] public [A-Za-z0-9]+;" #Array

  fixA1 = r"(uint|string|int)\[[0-9]+\]" #Fixed
  pat6 = r"[A-Za-z0-9]+\[\]" #Dynamic array

  pat4 = r"public [A-Za-z0-9]+ = 0;" #Default value

  #Functions
  extfunc = r"function [A-Za-z0-9]+\([^)]*\) external" #External function
  pubfunc = r"function [A-Za-z0-9]+\([^)]*\) public" #Pub funct

  def findCriteria(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

  def checkCriteria(pattern):
    return re.compile(pattern, re.IGNORECASE).search


  #Init settings
  benchmark_score = 0.0

  indivExcessive = []
  indivScores = []

  perfect_score = 100.0
  partial_score = 50.0
  no_score = 0.0

  #Information Tracking
  completeSC1 = 0
  completeSC2 = 0
  completeSC3 = 0
  storageType1 = 0
  storageType2 = 0
  storageType3 = 0
  packingVar1 = 0
  packingVar2 = 0
  packingBool1 = 0
  packingBool2 = 0
  defaultVal1 = 0
  defaultVal2 = 0
  dataType1 = 0
  dataType2 = 0
  dataType3 = 0
  sizeType1 = 0
  sizeType2 = 0
  functionType1 = 0
  functionType2 = 0
  functionType3 = 0
  metCriteria = 0
  exceedData = 0

  numCycle5 = 0
  for sc in smartContracts2:
    excessiveData = 0
    temp_score = 0.0

    #Complete SC
    if findCriteria(criteria2_ZONK)(sc):
      completeSC1 += 1
      temp_score += no_score
    elif findCriteria(criteria1_UFSC)(sc):
      completeSC2 += 1
      temp_score += (partial_score * cWeights[0])
    else:
      completeSC3 += 1
      temp_score += (perfect_score * cWeights[0])

    #Packing Variables
    if checkCriteria(notPacked)(sc):
      packingVar1 += 1
      temp_score += no_score
    else:
      packingVar2 += 1
      temp_score += (perfect_score * cWeights[1])

    #Packing Boolean
    if checkCriteria(isBoolean)(sc):
      packingBool1 += 1
      temp_score += (perfect_score * cWeights[2])
    else:
      packingBool2 += 1
      temp_score += no_score

    #Storage type
    if checkCriteria(pat7)(sc) and findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc): # Variety uint
      storageType1 += 1
      temp_score += (partial_score * cWeights[3])
    elif findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc) and not checkCriteria(pat7)(sc): # uint256
      storageType2 += 1
      temp_score += no_score
    elif checkCriteria(pat7)(sc) and not findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc): # uint*
      storageType3 += 1
      temp_score += (perfect_score * cWeights[3])

    #Mapping vs array
    if checkCriteria(pat1)(sc) and checkCriteria(pat3)(sc): # Mapping & Array
      dataType1 += 1
      temp_score += (partial_score * cWeights[4])
    elif checkCriteria(pat1)(sc) and not checkCriteria(pat3)(sc): # Mapping
      dataType2 += 1
      temp_score += (perfect_score * cWeights[4])
    elif checkCriteria(pat3)(sc) and not checkCriteria(pat1)(sc): # Array
      dataType3 += 1
      temp_score += no_score

    #Variable size type
    if checkCriteria(fixA1)(sc): # Fixed
      sizeType1 += 1
      temp_score += (perfect_score * cWeights[5])
    else: # Dynamic
      sizeType2 += 1
      temp_score += no_score

    #Default value
    if checkCriteria(pat4)(sc): # Initialized value
      defaultVal1 += 1
      temp_score += no_score
    else:
      defaultVal2 += 1
      temp_score += (perfect_score * cWeights[6]) # Default

    #Function
    if checkCriteria(extfunc)(sc) and checkCriteria(pubfunc)(sc): #mixed function
      functionType1 += 1
      temp_score += (partial_score * cWeights[7])
    elif checkCriteria(extfunc)(sc) and not checkCriteria(pubfunc)(sc): # only external
      functionType2 += 1
      temp_score += (perfect_score * cWeights[7])
    elif checkCriteria(pubfunc)(sc) and not checkCriteria(extfunc)(sc): # only public
      functionType3 += 1
      temp_score += no_score

    #Limit Storage & On-Chain data
    if chooseERC == "1":
      #ERC20 Criterias
      erc20_1 = "function name\([^)]*\)"
      erc20_2 = "function symbol\([^)]*\)"
      erc20_3 = "function decimal\([^)]*\)"
      erc20_4 = "function totalSupply\([^)]*\)"
      erc20_5 = "function balanceOf\([^)]*\)"
      erc20_6 = "function allowance\([^)]*\)"
      erc20_7 = "function transfer\([^)]*\)"
      erc20_8 = "function approve\([^)]*\)"
      erc20_9 = "function transferFrom\([^)]*\)"
      erc20_10 = "event Transfer\([^)]*\)"
      erc20_11 = "event Approval\([^)]*\)"

      offchains1 = "function (?!totalSupply|_totalSupply|balanceOf|_balanceOf|allowance|_allowance|transfer|_transfer|approve|_approve|transferFrom|_transferFrom|decimal|_decimal|symbol|_symbol|name|_name)\w+\([^)]*\)"
      offchains2 = "event (?!Approval|Transfer)\w+\([^)]*\)"

      tS = 0
      regexListSC1 = [erc20_1, erc20_2, erc20_3, erc20_4, erc20_5, erc20_6, erc20_7, erc20_8, erc20_9, erc20_10, erc20_11]

      erc20score = 100 / len(regexListSC1)
      ERC20criteriaMet = 0

      for testCase in regexListSC1:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc20score
          ERC20criteriaMet += 1

      if ERC20criteriaMet == len(regexListSC1):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])

    elif chooseERC == "2":
      #ERC4626 Criterias
      erc4626_1 = "function asset\([^)]*\)"
      erc4626_2 = "function totalAssets\([^)]*\)"
      erc4626_3 = "function convertToShares\([^)]*\)"
      erc4626_4 = "function convertToAssets\([^)]*\)"
      erc4626_5 = "function maxDeposit\([^)]*\)"
      erc4626_6 = "function previewDeposit\([^)]*\)"
      erc4626_7 = "function deposit\([^)]*\)"
      erc4626_8 = "function maxMint\([^)]*\)"
      erc4626_9 = "function previewMint\([^)]*\)"
      erc4626_10 = "function mint\([^)]*\)"
      erc4626_11 = "function maxWithdraw\([^)]*\)"
      erc4626_12 = "function previewWithdraw\([^)]*\)"
      erc4626_13 = "function withdraw\([^)]*\)"
      erc4626_14 = "function maxRedeem\([^)]*\)"
      erc4626_15 = "function previewRedeem\([^)]*\)"
      erc4626_16 = "function redeem\([^)]*\)"
      erc4626_17 = "event Deposit\([^)]*\)"
      erc4626_18 = "event Withdraw\([^)]*\)"

      offchains1 = "function (?!asset|totalAssets|convertToShares|convertToAssets|maxDeposit|previewDeposit|deposit|maxMint|previewMint|mint|maxWithdraw|previewWithdraw|withdraw|maxRedeem|previewRedeem|redeem)\w+\([^)]*\)"
      offchains2 = "event (?!Deposit|Withdraw)\w+\([^)]*\)"

      tS = 0
      regexListSC2 = [erc4626_1, erc4626_2, erc4626_3, erc4626_4, erc4626_5, erc4626_6, erc4626_7, erc4626_8, erc4626_9, erc4626_10, erc4626_11, erc4626_12, erc4626_13, erc4626_14, erc4626_15, erc4626_16, erc4626_17, erc4626_18]

      erc4626score = 100 / len(regexListSC2)
      ERC4626criteriaMet = 0

      for testCase in regexListSC2:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc4626score
          ERC4626criteriaMet += 1

      if ERC4626criteriaMet == len(regexListSC2):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])

    elif chooseERC == "3":
      erc3156_1 = "function maxFlashLoan\([^)]*\)"
      erc3156_2 = "function flashFee\([^)]*\)"
      erc3156_3 = "function flashLoan\([^)]*\)"
      erc3156_4 = "function onFlashLoan\([^)]*\)"

      offchains1 = "function (?!maxFlashLoan|flashFee|flashLoan|onFlashLoan)\w+\([^)]*\)"
      offchains2 = "event [A-Za-z0-9]+\([^)]*\)"

      tS = 0
      regexListSC3 = [erc3156_1, erc3156_2, erc3156_3, erc3156_4]

      erc3156score = 100 / len(regexListSC3)
      ERC3156criteriaMet = 0

      for testCase in regexListSC3:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc3156score
          ERC3156criteriaMet += 1

      if ERC3156criteriaMet == len(regexListSC3):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])


    indivExcessive.append(excessiveData)
    indivScores.append(round(temp_score, 2))
    numCycle5 += 1

    #Set final score
    benchmark_score += temp_score

  #Information Output
  print(f"[BENCHMARKING {numCycle5} SMART CONTRACTS RESULT]\n")
  print("Individual excessive data count:")
  print(', '.join([str(x) for x in indivExcessive]))
  print("Individual score:")
  print(', '.join([str(x) for x in indivScores]))
  print("\n")

  print("Complete source code")
  print("==============================")
  print(f"Complete SC: {completeSC3}")
  print(f"Unfinished SC: {completeSC2}")
  print(f"Failed SC: {completeSC1}\n")
  print("Packing variable")
  print("==============================")
  print(f"Packed Variable: {packingVar2}")
  print(f"Unpacked Variable: {packingVar1}\n")
  print("Packing boolean")
  print("==============================")
  print(f"Packed Bool: {packingBool1}")
  print(f"Unpacked Bool: {packingBool2}\n")
  print("Storage type")
  print("==============================")
  print(f"uint*: {storageType3}")
  print(f"Mixed storage: {storageType1}")
  print(f"uint256: {storageType2}\n")
  print("Data type")
  print("==============================")
  print(f"Only mapping: {dataType2}")
  print(f"Mixed: {dataType1}")
  print(f"Only array: {dataType3}\n")
  print("Variable size type")
  print("==============================")
  print(f"Fixed: {sizeType1}")
  print(f"Dynamic: {sizeType2}\n")
  print("Default value")
  print("==============================")
  print(f"Default Value: {defaultVal2}")
  print(f"Initialized: {defaultVal1}\n")
  print("Functions")
  print("==============================")
  print(f"External: {functionType2}")
  print(f"Mixed: {functionType1}")
  print(f"Public: {functionType3}\n")
  print("Storage limiting")
  print("==============================")
  print(f"Met storage standard: {metCriteria}\n")
  excessiveData /= 20
  print("Minimize on-chain data")
  print("==============================")
  print(f"Number of SC that exceed maximum on-chain data: {exceedData}\n")

  #Calculate average score
  print("============================================================")
  print(f"Total score: {round(benchmark_score, 2)}")
  benchmark_score /= len(smartContracts)
  print(f"Average score: {round(benchmark_score, 2)}")

benchmarkProgram()

[BENCHMARKING 5 SMART CONTRACTS RESULT]

Individual excessive data count:
4, 2, 0, 0, 1
Individual score:
57.36, 48.0, 58.64, 49.55, 55.82


Complete source code
Complete SC: 5
Unfinished SC: 0
Failed SC: 0

Packing variable
Packed Variable: 5
Unpacked Variable: 0

Packing boolean
Packed Bool: 0
Unpacked Bool: 5

Storage type
uint*: 0
Mixed storage: 4
uint256: 1

Data type
Only mapping: 4
Mixed: 1
Only array: 0

Variable size type
Fixed: 0
Dynamic: 5

Default value
Default Value: 3
Initialized: 2

Functions
External: 1
Mixed: 1
Public: 3

Storage limiting
Met storage standard: 0

Minimize on-chain data
Number of SC that exceed maximum on-chain data: 0

Total score: 269.36
Average score: 53.87
